In [7]:
from use_dataset import UseDataset
from deeplabv3 import DeepLabV3
from torch import no_grad, sigmoid
from deeplabv3 import DeepLabV3
from unet import UNet
from unetplusplus import UNetPlusPlus
from dataset import CVDataset
from metrics import IoU_torch, f1_scores
from inference_dataset import CVInferenceDataset

from torch.utils.data import DataLoader
from os import cpu_count
from torch import no_grad
from torch.cuda import is_available as cuda_is_available
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import torch
from torch import tensor, cuda, is_tensor, float32, long, from_numpy
import matplotlib.pyplot as plt
import gc
from numpy import array
import os
import shutil
import numpy as np
from torch.utils.data import DataLoader
from os import cpu_count
from cv2 import imread, imwrite
import torch

data = UseDataset('/home/jupyter/datasphere/project/satellite_hack/', (1024,1024))

model = DeepLabV3(2)
model.load_state_dict(torch.load('/home/jupyter/datasphere/project/satellite_hack/model_deeplab.pth'))
model = model.to('cuda')

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /tmp/xdg_cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100%|██████████| 233M/233M [00:03<00:00, 72.4MB/s] 


In [10]:
device = 'cuda' if cuda_is_available() else 'cpu'
batch_size = 3
model.eval()
matrix_shape = (1024,1024)
with no_grad():
    for k in range(len(data)):
        image_shape, dat = data[k]
        count_n = len(dat)
        loader = DataLoader(dat, batch_size=batch_size, shuffle=False, num_workers=cpu_count(), pin_memory=False)
        part_output = []
        for x in loader:
            images = x.to(device)
            output = (model(images).softmax(1)[:, 0] > 0.7).detach().cpu().numpy()
            for j in range(len(output)):
                part_output.append(output[j])
        part_output = np.array(part_output)

        reconstructed_image = np.zeros(image_shape, dtype=np.uint8)
        h, w = image_shape
        for i in range(0, reconstructed_image.shape[0], matrix_shape[0]):
            for j in range(0, reconstructed_image.shape[1], matrix_shape[1]):
                image_part_tensor = part_output[0]
                image_part = (image_part_tensor * 255).astype(np.uint8)
                image_part = image_part[:matrix_shape[0], :matrix_shape[1]]
                try:
                    reconstructed_image[i:i+matrix_shape[0], j:j+matrix_shape[1]] = image_part
                except:
                    reconstructed_image[i:i+matrix_shape[0], j:j+matrix_shape[1]] = image_part[:h - i, :w - j]
                try:
                    part_output = part_output[1:]
                except:
                    pass

        imwrite(f'../masks/test_mask_00{k}.png', reconstructed_image)